In [27]:
import pandas as pd

data = pd.read_csv('/content/covid_19_clean_complete.csv')

print(data.info())
print(data.head())


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 49068 entries, 0 to 49067
Data columns (total 10 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   Province/State  14664 non-null  object 
 1   Country/Region  49068 non-null  object 
 2   Lat             49068 non-null  float64
 3   Long            49068 non-null  float64
 4   Date            49068 non-null  object 
 5   Confirmed       49068 non-null  int64  
 6   Deaths          49068 non-null  int64  
 7   Recovered       49068 non-null  int64  
 8   Active          49068 non-null  int64  
 9   WHO Region      49068 non-null  object 
dtypes: float64(2), int64(4), object(4)
memory usage: 3.7+ MB
None
  Province/State Country/Region       Lat       Long        Date  Confirmed  \
0            NaN    Afghanistan  33.93911  67.709953  2020-01-22          0   
1            NaN        Albania  41.15330  20.168300  2020-01-22          0   
2            NaN        Algeria  28.03390   1.6

In [28]:
data.isnull().sum()

,0
Province/State,34404
Country/Region,0
Lat,0
Long,0
Date,0
Confirmed,0
Deaths,0
Recovered,0
Active,0
WHO Region,0


In [29]:
data['Province/State'].fillna('Unknown', inplace=True)


<ipython-input-29-7633d9d010c3>:1: FutureWarning:

A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.





In [30]:
data.isnull().sum()

,0
Province/State,0
Country/Region,0
Lat,0
Long,0
Date,0
Confirmed,0
Deaths,0
Recovered,0
Active,0
WHO Region,0


In [34]:
data['Date'] = pd.to_datetime(data['Date'])

In [35]:
global_daily = data.groupby('Date').sum().reset_index()


In [36]:
country_data = data.groupby(['Date', 'Country/Region']).sum().reset_index()


In [37]:
print(data.info())
print(data.head())


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 49068 entries, 0 to 49067
Data columns (total 10 columns):
 #   Column          Non-Null Count  Dtype         
---  ------          --------------  -----         
 0   Province/State  49068 non-null  object        
 1   Country/Region  49068 non-null  object        
 2   Lat             49068 non-null  float64       
 3   Long            49068 non-null  float64       
 4   Date            49068 non-null  datetime64[ns]
 5   Confirmed       49068 non-null  int64         
 6   Deaths          49068 non-null  int64         
 7   Recovered       49068 non-null  int64         
 8   Active          49068 non-null  int64         
 9   WHO Region      49068 non-null  object        
dtypes: datetime64[ns](1), float64(2), int64(4), object(3)
memory usage: 3.7+ MB
None
  Province/State Country/Region       Lat       Long       Date  Confirmed  \
0        Unknown    Afghanistan  33.93911  67.709953 2020-01-22          0   
1        Unknown        A

In [ ]:

data.fillna(0, inplace=True)  # Replace other NaNs with zeros


In [ ]:
data['Mortality Rate'] = (data['Deaths'] / data['Confirmed']) * 100


In [ ]:
import plotly.express as px

global_data = data.groupby('Date').sum().reset_index()

fig = px.line(global_data, x='Date', y=['Confirmed', 'Deaths', 'Recovered'],
              title='Global COVID-19 Trends')
fig.show()


In [ ]:
country_specific = country_data[country_data['Country/Region'] == 'United States']

fig = px.line(country_specific, x='Date', y=['Confirmed', 'Deaths', 'Recovered'],
              title='COVID-19 Trends in United States')
fig.show()


In [ ]:
latest_data = data[data['Date'] == data['Date'].max()]

fig = px.bar(latest_data, x='Country/Region', y='Mortality Rate',
             title='Mortality Rate by Country')
fig.show()


In [ ]:
pip install dash


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 43.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 228.0/228.0 kB 14.9 MB/s eta 0:00:00
  Attempting uninstall: Werkzeug
    Found existing installation: Werkzeug 3.1.3
    Uninstalling Werkzeug-3.1.3:
      Successfully uninstalled Werkzeug-3.1.3


In [38]:
from dash import Dash, html, dcc
from dash.dependencies import Input, Output
import pandas as pd
import plotly.graph_objs as go

# Sample data
data = pd.read_csv('/content/covid_19_clean_complete.csv')
data['Date'] = pd.to_datetime(data['Date'])
data['Province/State'].fillna('Unknown', inplace=True)

# Dash app
app = Dash(__name__)

# Layout
app.layout = html.Div([
    html.H1('COVID-19 Dashboard'),
    dcc.Dropdown(
        id='country-dropdown',
        options=[{'label': country, 'value': country} for country in data['Country/Region'].unique()],
        value='United States'  # Default selection
    ),
    dcc.Graph(id='country-trends')
])

# Callback
@app.callback(
    Output('country-trends', 'figure'),
    [Input('country-dropdown', 'value')]
)
def update_trends(selected_country):
    country_data = data[data['Country/Region'] == selected_country]
    figure = {
        'data': [
            go.Scatter(x=country_data['Date'], y=country_data['Confirmed'], mode='lines', name='Confirmed'),
            go.Scatter(x=country_data['Date'], y=country_data['Deaths'], mode='lines', name='Deaths'),
            go.Scatter(x=country_data['Date'], y=country_data['Recovered'], mode='lines', name='Recovered'),
        ],
        'layout': {'title': f'COVID-19 Trends in {selected_country}'}
    }
    return figure

if __name__ == '__main__':
    app.run_server(debug=True)


<ipython-input-38-47ddbdc69205>:9: FutureWarning:

A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.





<IPython.core.display.Javascript object>